In [3]:
!pip uninstall antlr4-python3-runtime -y
!pip install antlr4-python3-runtime==4.13.1

Found existing installation: antlr4-python3-runtime 4.9.3
Uninstalling antlr4-python3-runtime-4.9.3:
  Successfully uninstalled antlr4-python3-runtime-4.9.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 8.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
omegaconf 2.3.0 requires antlr4-python3-runtime==4.9.*, but you have antlr4-python3-runtime 4.13.1 which is incompatible.


In [4]:
# Descarga la versión compatible de ANTLR
!curl -O https://www.antlr.org/download/antlr-4.13.1-complete.jar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2089k  100 2089k    0     0  8773k      0 --:--:-- --:--:-- --:--:-- 8777k


In [5]:
import numpy as np
import pandas as pd
from antlr4 import *

In [6]:
# Definición de la tabla de símbolos como un diccionario
class SymbolTable:
    def __init__(self):
        self.symbols = {}

    def declare(self, id, type_, value):
        """Declarar una nueva variable en la tabla de símbolos"""
        self.symbols[id] = {'type': type_, 'value': value}

    def get_value(self, id):
        """Obtener el valor de una variable"""
        return self.symbols.get(id, {}).get('value', None)

    def update(self, id, value):
        """Actualizar el valor de una variable"""
        if id in self.symbols:
            self.symbols[id]['value'] = value
        else:
            raise ValueError(f"Variable '{id}' no definida")

    def __str__(self):
        return str(self.symbols)



In [7]:
# Crear una instancia de la tabla de símbolos
symbol_table = SymbolTable()

# Declaración de variables (ejemplo)
symbol_table.declare('horas_trabajadas', 'float', 180)
symbol_table.declare('proyectos_completados', 'int', 3)
symbol_table.declare('errores_cometidos', 'int', 2)
symbol_table.declare('evaluacion_supervisor', 'float', 7.5)
symbol_table.declare('puntualidad', 'float', 85)

In [8]:
# Funciones estadísticas

def calcular_media(*args):
    """Calcular la media de un conjunto de valores"""
    return np.mean(args)

def calcular_varianza(*args):
    """Calcular la varianza de un conjunto de valores"""
    return np.var(args)

def calcular_coef_fisher(varianza_1, varianza_2):
    """Calcular el coeficiente de Fisher"""
    return varianza_1 / varianza_2 if varianza_2 != 0 else None


In [9]:
# Calcular las estadísticas para las variables en la tabla de símbolos
horas_trabajadas = symbol_table.get_value('horas_trabajadas')
proyectos_completados = symbol_table.get_value('proyectos_completados')
errores_cometidos = symbol_table.get_value('errores_cometidos')
evaluacion_supervisor = symbol_table.get_value('evaluacion_supervisor')
puntualidad = symbol_table.get_value('puntualidad')

In [10]:
# Imprimir la tabla de símbolos
print("\nTabla de símbolos:")
print(symbol_table)


Tabla de símbolos:
{'horas_trabajadas': {'type': 'float', 'value': 180}, 'proyectos_completados': {'type': 'int', 'value': 3}, 'errores_cometidos': {'type': 'int', 'value': 2}, 'evaluacion_supervisor': {'type': 'float', 'value': 7.5}, 'puntualidad': {'type': 'float', 'value': 85}}


EJEMPLO CON ANTLR

In [11]:
grammar = """
grammar EmployeeEval;

// Reglas del Parser
program      : (declaration | function)+ EOF;
declaration  : TYPE ID '=' expr ';' ;
function     : FUNC TYPE ID '(' paramList? ')' block ;
paramList    : param (',' param)* ;
param        : TYPE ID ;
block        : '{' statement* '}' ;
statement    : expr ';'
             | declaration
             | COMMENT ;
expr         : ID                                // Variable
             | NUMBER                            // Número constante
             | STRING                            // Cadena
             | expr '+' expr                     // Suma
             | expr '-' expr                     // Resta
             | expr '*' expr                     // Multiplicación
             | expr '/' expr                     // División
             | 'mean' '(' expr (',' expr)* ')'    // Media
             | 'variance' '(' expr (',' expr)* ')' // Varianza
             | 'stddev' '(' expr (',' expr)* ')'   // Desviación estándar
             | 'fisher' '(' expr (',' expr)* ')'   // Coeficiente de Fisher
             ;


TYPE         : 'int' | 'float' | 'string' ;     // Tipos de datos
FUNC         : 'func' ;                         // Definición de funciones
ID           : [a-zA-Z_][a-zA-Z_0-9]* ;         // Identificadores (variables y funciones)
NUMBER       : [0-9]+ ('.' [0-9]+)? ;           // Números (enteros o decimales)
STRING       : '"' .*? '"' ;                    // Cadenas de texto
WS          : [ \\t\\r\\n]+ -> skip ;
COMMENT : \'//\' ~[\\r\\n]* -> skip ;
"""
with open("EmployeeEval.g4", "w") as f:
  f.write(grammar)


In [12]:
# Regenera los parsers
!java -jar antlr-4.13.1-complete.jar -Dlanguage=Python3 -listener -visitor EmployeeEval.g4

In [13]:
input_data = """
float horas_trabajadas = 180;
int proyectos_completados = 3;
int errores_cometidos = 2;
float evaluacion_supervisor = 7.5;
float puntualidad = 85;

mean(horas_trabajadas, proyectos_completados, errores_cometidos, evaluacion_supervisor);
variance(horas_trabajadas, proyectos_completados, errores_cometidos, evaluacion_supervisor);
stddev(horas_trabajadas, proyectos_completados, errores_cometidos, evaluacion_supervisor);
fisher(horas_trabajadas, proyectos_completados, errores_cometidos, evaluacion_supervisor);

"""


In [14]:
# Ahora importar los módulos generados
from EmployeeEvalLexer import EmployeeEvalLexer
from EmployeeEvalParser import EmployeeEvalParser
from EmployeeEvalListener import EmployeeEvalListener

# Definición mejorada de la tabla de símbolos
class SymbolTable:
    def __init__(self):
        self.symbols = {}

    def declare(self, id, type_, value=None):
        """Declarar una nueva variable en la tabla de símbolos"""
        self.symbols[id] = {'type': type_, 'value': value}

    def get_value(self, id):
        """Obtener el valor de una variable"""
        return self.symbols.get(id, {}).get('value', None)

    def update(self, id, value):
        """Actualizar el valor de una variable"""
        if id in self.symbols:
            self.symbols[id]['value'] = value
        else:
            raise ValueError(f"Variable '{id}' no definida")

    def __str__(self):
        return "\n".join([f"{name}: {details['type']} = {details['value']}"
                         for name, details in self.symbols.items()])


In [15]:
# Listener mejorado
class EvalListener(EmployeeEvalListener):
    def __init__(self):
        self.symbol_table = SymbolTable()
        self.current_value = None

    def enterDeclaration(self, ctx):
        var_name = ctx.ID().getText()
        var_type = ctx.TYPE().getText()
        value = self.evaluate_expr(ctx.expr())
        self.symbol_table.declare(var_name, var_type, value)

    def evaluate_expr(self, ctx):
        if ctx.getChildCount() == 1:  # Número, variable o cadena
            child = ctx.getChild(0)
            if isinstance(child, TerminalNode):
                text = child.getText()
                if child.getSymbol().type == EmployeeEvalLexer.NUMBER:
                    return float(text) if '.' in text else int(text)
                elif child.getSymbol().type == EmployeeEvalLexer.ID:
                    return self.symbol_table.get_value(text)
                elif child.getSymbol().type == EmployeeEvalLexer.STRING:
                    return text.strip('"')
        elif ctx.getChildCount() == 3:  # Operación binaria o paréntesis
            if ctx.getChild(0).getText() == '(':
                return self.evaluate_expr(ctx.getChild(1))
            else:
                left = self.evaluate_expr(ctx.getChild(0))
                right = self.evaluate_expr(ctx.getChild(2))
                op = ctx.getChild(1).getText()
                if op == '+': return left + right
                if op == '-': return left - right
                if op == '*': return left * right
                if op == '/': return left / right
        elif ctx.getChildCount() > 3:  # Función estadística
            func_name = ctx.getChild(0).getText()
            args = []
            for i in range(2, ctx.getChildCount()-1, 2):  # Saltar comas
                args.append(self.evaluate_expr(ctx.getChild(i)))

            if func_name == 'mean':
                return np.mean(args)
            elif func_name == 'variance':
                return np.var(args)
            elif func_name == 'stddev':
                return np.std(args)
            elif func_name == 'fisher':
                n = len(args)
                mean = np.mean(args)
                stddev = np.std(args)
                return np.sum([(x - mean)**3 for x in args]) / (n * stddev**3)
        return None

    def exitExpr(self, ctx):
        if ctx.parentCtx and isinstance(ctx.parentCtx, EmployeeEvalParser.StatementContext):
            result = self.evaluate_expr(ctx)
            if result is not None:
                print(f"Resultado: {result}")

In [16]:
# Procesamiento
lexer = EmployeeEvalLexer(InputStream(input_data))
tokens = CommonTokenStream(lexer)
parser = EmployeeEvalParser(tokens)
tree = parser.program()

listener = EvalListener()
walker = ParseTreeWalker()
walker.walk(listener, tree)

line 8:0 extraneous input 'mean' expecting {<EOF>, TYPE, 'func'}


In [17]:
# Mostrar resultados finales
print("\nTabla de símbolos final:")
print(listener.symbol_table)


Tabla de símbolos final:
horas_trabajadas: float = 180
proyectos_completados: int = 3
errores_cometidos: int = 2
evaluacion_supervisor: float = 7.5
puntualidad: float = 85
